# Binomial logistic regression retrospect

In previous post [Logistic regression (binomial regression) and regularization](https://lnshi.github.io/ml-exercises/ml_basics_in_html/rdm007_logistic_regression%28binomial_regression%29_and_regularization/logistic_regression%28binomial_regression%29_and_regularization.html#Modeling) we revealed the model for logistic regression directly: $h_\theta(x) = \frac{1}{1+e^{-\theta x}}$, for why the model looks like that we already had one explanation in the post: [GLM and exponential family distributions](https://lnshi.github.io/ml-exercises/ml_basics_in_html/rdm008_GLM_and_exponential_family_distributions/GLM_and_exponential_family_distributions.html#With-above-three-hypotheses,-GLM-$\Rightarrow$-logistic-regression), in this post lets interpret it in another way.

Logistic regression is inspired from linear regression: $h_\theta(x) = \theta x$, but to a binary classifier(binomial logistic regression) we hope the corresponding $\theta x$ part can indicate a probability: the probability ($p$) of the sample point belongs to class $A$ (then for $\bar{A} \text{ is } 1-p$), since $p$ is a probability, then its range should be $[0,1]$, but the reality is $\theta x$ can take any value, for achieving what we want we can introduce in [odds](https://en.wikipedia.org/wiki/Odds):

$$
\text{odds } = \frac{p}{1-p}
$$

and log-**it** (it → odds, log-__odds__):

$$
\begin{align*}
  ln\frac{p}{1-p} &= \theta x \\
  \Rightarrow p &= \frac{e^{\theta x}}{1 + e^{\theta x}} = \frac{1}{1 + e^{-\theta x}}
\end{align*}
$$

now $p \in [0, 1]$, that is: when we do the $\frac{1}{1+e^{-\theta x}}$ transformation to $x$ we get probabilities, and then we can use the odds/log-odds, that is why the model looks like that!

# Extend binomial logistic regression to multinomial logistic regression

For binomial logistic regression we only have two classes: $A \text{ and } \bar{A}$, then we can use the log-odds $ln\frac{p}{1-p}$ as the binomial classifier indicator: $> 0 \text{ belongs to class A, } < 0 \text{ belongs to class } \bar{A}$ but how do we deal with the case that we have more than two classes, how do we extend the log-odds indicator?

Lets say the sample sapce can be classified into $k$ classes, for solving above question, we can go with the below approach:

We construct amount $k$ classifiers, each of these $k$ classifiers just does exactly same stuff like the previous [binomial logistic regression](https://lnshi.github.io/ml-exercises/ml_basics_in_html/rdm007_logistic_regression%28binomial_regression%29_and_regularization/logistic_regression%28binomial_regression%29_and_regularization.html#How-to-estimate-the-$\theta$:-MLE-(Maximum-Likelihood-Estimation)) does: evaluate each of the training sample point between class $k_i$ and $\bar{k_i}$, that is evaluating the $ln\frac{p_{k_i}}{p_{\bar{k_i}}}$.

Lets say we have 5 samples: $x_0, x_1, x_2, x_3, x_4$, and they can be classified into 3 classes: $k_0, k_1, k_2$, and lets define $p_{k_i}^{x_j}$ represents the probability of that sample $x_j$ belongs to class $k_i$, then after running through this approach eventually we will get:

$$
\begin{pmatrix}
  p_{k_0}^{x_0} & p_{k_1}^{x_0} & p_{k_2}^{x_0} \\
  p_{k_0}^{x_1} & p_{k_1}^{x_1} & p_{k_2}^{x_1} \\
  p_{k_0}^{x_2} & p_{k_1}^{x_2} & p_{k_2}^{x_2} \\
  p_{k_0}^{x_3} & p_{k_1}^{x_3} & p_{k_2}^{x_3} \\
  p_{k_0}^{x_4} & p_{k_1}^{x_4} & p_{k_2}^{x_4}
\end{pmatrix}
$$

At last, we just pick up the class has the most probability the input belongs to as our predicition result (from each row)!

# Hand-written digits recognition with multinomial logistic regression

In [81]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sets the backend of matplotlib to the 'inline' backend.
#
# With this backend, the output of plotting commands is displayed inline within frontends like the Jupyter notebook,
# directly below the code cell that produced it.
# The resulting plots will then also be stored in the notebook document.
#
# More details: https://stackoverflow.com/questions/43027980/purpose-of-matplotlib-inline
%matplotlib inline

from scipy.io import loadmat

data = loadmat(os.getcwd() + '/hand_written_digits.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [82]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [83]:
X = np.insert(data['X'], 0, values=np.ones(data['X'].shape[0]), axis=1)
y = data['y']
X.shape, y.shape

((5000, 401), (5000, 1))

The above data contains amount 5000 of hand-written digits, and each single digit holds a 20 by 20 pixels grid, that is each row of above $X$ represents one digit, and each of its component is a float number which represents the grayscale intensity of one of the 20*20 pixels.

And we also noticed that the value of $y \in \{1,2,3,4,5,6,7,8,9,10\}$, so the value of $y$ is not the actual number for that corresponding row of $X$, but a class label: class 1, class 2, ... , class 10.

Partial example of the data:

<img src="./hand_written_digits.png">

In [84]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### `cost_reg` is exactly copied over from previous post: [New cost function with regularization item](https://lnshi.github.io/ml-exercises/ml_basics_in_html/rdm007_logistic_regression%28binomial_regression%29_and_regularization/logistic_regression%28binomial_regression%29_and_regularization.html#New-cost-function-with-regularization-item)

In [85]:
def cost_reg(theta, X, y, alpha):
    theta = np.reshape(theta, (-1, len(theta)))

    assert X.shape[1] == theta.shape[1], \
      'Improper shape of theta, expected to be: {}, actual: {}'.format((1, X.shape[1]), theta.shape)

    part0 = np.multiply(y, np.log(sigmoid(X @ theta.T)))
    part1 = np.multiply(1 - y, np.log(1 - sigmoid(X @ theta.T)))
    reg = alpha / (2 * len(X)) * np.sum(np.power(theta[:, 1:theta.shape[1]], 2))

    return -np.sum(part0 + part1) / len(X) + reg

### `gradient_reg` exactly does what the previous [gradient_reg](https://lnshi.github.io/ml-exercises/ml_basics_in_html/rdm007_logistic_regression%28binomial_regression%29_and_regularization/logistic_regression%28binomial_regression%29_and_regularization.html#New-gradient-function-with-regularization-item) does, just we replace the previous for loop calculating way with current pure matrix operating way (then reset grad[0,0] afterwards)

In [86]:
def gradient_reg(theta, X, y, alpha):
    theta = np.reshape(theta, (-1, len(theta)))

    assert X.shape[1] == theta.shape[1], \
      'Improper shape of theta, expected to be: {}, actual: {}'.format((1, X.shape[1]), theta.shape)

    error = sigmoid(X @ theta.T) - y
    
    grad = ((X.T @ error) / len(X)).T + alpha / len(X) * theta
    
    # Reset grad[0,0] to make the intercept gradient is not regularized.
    grad[0, 0] = np.sum(np.multiply(error, X[:, [0]])) / len(X)

    return grad.ravel()

### Evaluate each of the training sample point between class $k_i$ and $\bar{k_i}$, that is evaluating the $ln\frac{p_{k_i}}{p_{\bar{k_i}}}$

In [87]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, alpha):
    # 'k by (n+1), n=20*20 here' array for the parameters of each of the k classifiers.
    all_theta = np.zeros((num_labels, X.shape[1]))
    
    # Labels are 1-indexed instead of 0-indexed, it is decided by the data `y`.
    for i in range(1, num_labels + 1):
        theta = np.zeros(X.shape[1])
        
        # Set the components of y which are class `i` as 1, all others as 0.
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (X.shape[0], 1))
        
        # Minimize the objective function.
        fmin = minimize(fun=cost_reg, x0=theta, args=(X, y_i, alpha), method='TNC', jac=gradient_reg)
        
        all_theta[i-1, :] = fmin.x
        
    return all_theta

In [88]:
all_theta = one_vs_all(X, y, 10, 1)
all_theta.shape

(10, 401)

In [89]:
def predict_all(X, all_theta):
    # Compute the class probability for each class for each training instance.
    h = sigmoid(X @ all_theta.T)
    
    # Generate array of the index with the maximum probability for each training instance.
    h_argmax = np.argmax(h, axis=1)
    
    # Because here the array is zero-indexed, and our class labels are 1-indexed,
    # we need to add one to get the real label prediction.
    return h_argmax + 1

In [90]:
y_pred = predict_all(X, all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = sum(map(int, correct)) / float(len(correct))
print('Total accuracy: {0:.2f}%'.format(accuracy * 100))

Total accuracy: 94.46%


# References

- [机器学习练习（四）——多元逻辑回归](https://blog.csdn.net/and_w/article/details/53260460)

- [Multinomial Logistic Regression](https://blog.csdn.net/baimafujinji/article/details/51703322)

- [Machine Learning Exercise 3 - Multi-Class Classification](https://github.com/jdwittenauer/ipython-notebooks/blob/master/notebooks/ml/ML-Exercise3.ipynb)